<a href="https://colab.research.google.com/github/Brian0615/CSC413FinalProject/blob/brian/HMSA_Sensitivity_Analysis/HMSA_Sensitivity_Analysis_Cityscapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HMSA - Sensitivity Analysis using the Cityscapes Dataset

## Overview
This file is used to reproducing results from the Hierarchical Multi-Scale Attention paper. Essentially, this file runs the evaluation script provided by the paper's authors as a simple test to confirm the results that the authors reported, and also whether we are able to use the code in Colab.

## Before Running the Code
1. Ensure that the runtime is set to GPU!
2. Ensure that the Colab VM is in the same region as the Google Cloud Storage Buckets (which are `US-multi-region`).
  - This minimizes costs, as Google charges for network egress.
  - The Colab region can be checked by running the cell below.
  - Perform a factory reset of the runtime until the runtime is in the US.

In [1]:
!curl ipinfo.io

{
  "ip": "35.231.119.142",
  "hostname": "142.119.231.35.bc.googleusercontent.com",
  "city": "North Charleston",
  "region": "South Carolina",
  "country": "US",
  "loc": "32.8546,-79.9748",
  "org": "AS15169 Google LLC",
  "postal": "29415",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

In [2]:
# check GPU specs
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-4fd11fa2-d14e-4b2e-7919-47976ec19d86)


## 1. Setup


### 1.1. Clone Git Repo

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/
!git clone https://github.com/Brian0615/CSC413FinalProject.git
%cd /content/drive/MyDrive/CSC413FinalProject/
!git checkout brian

/content/drive/MyDrive
Cloning into 'CSC413FinalProject'...
remote: Enumerating objects: 430, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 430 (delta 103), reused 349 (delta 63), pack-reused 0
Receiving objects: 100% (430/430), 22.06 MiB | 21.57 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Checking out files: 100% (272/272), done.
/content/drive/MyDrive/CSC413FinalProject
M	HMSA/train.py
M	PyTorch-Encoding/docs/source/_static/img/upconv.png
M	PyTorch-Encoding/tests/lint.py
Branch 'brian' set up to track remote branch 'brian' from 'origin'.
Switched to a new branch 'brian'


###1.2. Download Cityscapes Dataset

In [4]:
%cd /content
!mkdir cityscapes
%cd cityscapes
!mkdir leftImg8bit_trainvaltest
!mkdir gtFine_trainvaltest
%cd leftImg8bit_trainvaltest
!mkdir leftImg8bit
%cd leftImg8bit
!mkdir train
!mkdir val
!mkdir test

/content
/content/cityscapes
/content/cityscapes/leftImg8bit_trainvaltest
/content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit


In [3]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapi

In [6]:
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/gtFine /content/cityscapes/gtFine_trainvaltest/

Streaming output truncated to the last 5000 lines.
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_color.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_polygons.json...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_color.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_instanceIds.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_labelIds.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_polygons.json...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051877_gtFine_color.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051877_

In [7]:
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/train/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/val/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/test/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/

Streaming output truncated to the last 5000 lines.
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000005_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000007_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000008_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000009_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000010_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000012_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000011_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000015_000019_leftImg8bit.png...
Copying gs://csc413-final-pro

### 1.3. Download Weights


In [8]:
!mkdir /content/seg_weights/
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/ocrnet.HRNet_industrious-chicken.pth /content/seg_weights/
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth /content/seg_weights/

Copying gs://csc413-final-project-hmsa-weights/original_weights/ocrnet.HRNet_industrious-chicken.pth...
/ [1/1 files][807.0 MiB/807.0 MiB] 100% Done  69.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/807.0 MiB.                                    
Copying gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth...
\ [1/1 files][296.2 MiB/296.2 MiB] 100% Done                                    
Operation completed over 1 objects/296.2 MiB.                                    


In [9]:
!mkdir /content/uniform_centroids/
!gsutil -m cp gs://csc413-final-project-cityscapes-data/uniform_centroids/* /content/uniform_centroids/

Copying gs://csc413-final-project-cityscapes-data/uniform_centroids/cityscapes_cv0_tile1024.json...


### 1.4. Install Dependencies

In [10]:
# install basic dependencies
!pip install runx==0.0.6 numpy sklearn h5py jupyter scikit-image pillow piexif cffi tqdm dominate opencv-python nose ninja
!apt-get update
!apt-get install libgtk2.0-dev -y
!rm -rf /var/lib/apt/lists/*

     |████████████████████████████████| 163kB 30.6MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 645kB 52.2MB/s 
     |████████████████████████████████| 122kB 55.5MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://de

In [11]:
# change cuda version to 10.1
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda  # replace symlink to cuda-11.0 with cuda-10.1

/usr/local


In [12]:
# install apex
%cd /home/
!git clone https://github.com/NVIDIA/apex.git apex
%cd apex
!python setup.py install --cuda_ext --cpp_ext

/home
Cloning into 'apex'...
remote: Enumerating objects: 7963, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 7963 (delta 15), reused 8 (delta 0), pack-reused 7913
Receiving objects: 100% (7963/7963), 14.04 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (5414/5414), done.
/home/apex


torch.__version__  = 1.8.1+cu101


setup.py:67: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-i

### 1.5. Config Setup
 - Inside `CSC413FinalProject/HMSA/config.py`, set the following items:
  ```
  __C.ASSETS_PATH = '/content'
  __C.DATASET.CITYSCAPES_DIR = os.path.join(__C.ASSETS_PATH, 'cityscapes')
  ```

* Since Colab only has one GPU, we need to adjust the evaluation script
* Inside `CSC413FinalProject/HMSA/scripts/train_cityscapes_deepv3.yml`, set the following:
  ```
  CMD: "python -m torch.distributed.launch --nproc_per_node=1 train.py"
  ```
  (i.e. change `--nproc_per_node=8` to `--nproc_per_node=1`)

## 2. Training on Cityscapes

In [13]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=7bdebc7b3ce85fdb8f13ec29ccc5a6eda51374db0cacc4df501ff1e4e2014047
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  |     Proc size: 125.0 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [14]:
%cd /content/drive/MyDrive/CSC413FinalProject/HMSA/

/content/drive/MyDrive/CSC413FinalProject/HMSA


In [16]:
# dry run (to see full command)
!python -m runx.runx scripts/train_cityscapes.yml -i -n

python -m torch.distributed.launch --nproc_per_node=1 train.py --dataset cityscapes --cv 0 --syncbn --apex --fp16 --crop_size 100,200 --bs_trn 32 --poly_exp 2 --lr 5e-3 --rmi_loss --max_epoch 15 --n_scales 0.5,1.0,2.0 --supervised_mscale_loss_wt 0.05 --snapshot ASSETS_PATH/seg_weights/ocrnet.HRNet_industrious-chicken.pth --arch ocrnet.HRNet_Mscale --result_dir logs/train_cityscapes/ocrnet.HRNet_Mscale_authentic-vulture_2021.04.15_12.00 


In [17]:
# real run
!python -m runx.runx scripts/train_cityscapes.yml -i

None
Global Rank: 0 Local Rank: 0
Torch version: 1.8, 1.8.1+cu101
n scales [0.5, 1.0, 2.0]
dataset = cityscapes
ignore_label = 255
num_classes = 19
cv split val 0 ['val/lindau', 'val/munster', 'val/frankfurt']
mode val found 500 images
cn num_classes 19
cv split train 0 ['train/aachen', 'train/bochum', 'train/bremen', 'train/cologne', 'train/darmstadt', 'train/dusseldorf', 'train/erfurt', 'train/hamburg', 'train/hanover', 'train/jena', 'train/krefeld', 'train/monchengladbach', 'train/strasbourg', 'train/stuttgart', 'train/tubingen', 'train/ulm', 'train/weimar', 'train/zurich']
mode train found 2975 images
cn num_classes 19
Loading centroid file /content/uniform_centroids/cityscapes_cv0_tile1024.json
Found 19 centroids
Class Uniform Percentage: 0.5
Class Uniform items per Epoch: 2975
cls 0 len 5866
cls 1 len 5184
cls 2 len 5678
cls 3 len 1312
cls 4 len 1723
cls 5 len 5656
cls 6 len 2769
cls 7 len 4860
cls 8 len 5388
cls 9 len 2440
cls 10 len 4722
cls 11 len 3719
cls 12 len 1239
cls 13 l

In [4]:
# upload results to google cloud bucket
# !gsutil -m cp -r /content/uniform_centroids/* gs://csc413-final-project-cityscapes-data/uniform_centroids/
!gsutil -m cp -r /content/drive/MyDrive/CSC413FinalProject/HMSA/logs/train_cityscapes/* gs://csc413-final-project-hmsa-weights/training_results/

Copying file:///content/drive/MyDrive/CSC413FinalProject/HMSA/logs/train_cityscapes/ocrnet.HRNet_Mscale_busy-gecko_2021.04.14_21.52/hparams.json [Content-Type=application/json]...
Copying file:///content/drive/MyDrive/CSC413FinalProject/HMSA/logs/train_cityscapes/ocrnet.HRNet_Mscale_busy-gecko_2021.04.14_21.52/logging.log [Content-Type=application/octet-stream]...
Copying file:///content/drive/MyDrive/CSC413FinalProject/HMSA/logs/train_cityscapes/ocrnet.HRNet_Mscale_busy-gecko_2021.04.14_21.52/last_checkpoint_ep0.pth [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who down